### Preliminary Model (WSDM-KKBox Churn Prediction)

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.style.use('ggplot')
import re

** We're running our predictive model on data that we've further pre-processed on Alteryx Designer **

In [2]:
# data = pd.read_csv('train_transactions_clean.csv')
data = pd.read_csv('train_data.csv')
data.head(10)

,msno,city,bd,gender,registered_via,no_of_transactions,no_of_cancellations,cancelled,total_amount_paid,is_auto_renew,...,list_price_596,list_price_699,list_price_799,list_price_930,list_price_1000,list_price_1150,list_price_1200,list_price_1599,list_price_2000,is_churn
0,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,1.0,0.0,missing,7.0,4,0,0,396,True,...,0,0,0,0,0,0,0,0,0,False
1,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,15.0,26.0,male,9.0,19,0,0,2831,True,...,0,0,0,0,0,0,0,0,0,False
2,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,1.0,0.0,missing,7.0,26,0,0,3874,True,...,0,0,0,0,0,0,0,0,0,False
3,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,18.0,21.0,male,7.0,12,0,0,1788,True,...,0,0,0,0,0,0,0,0,0,False
4,++/UDNo9DLrxT8QVGiDi1OnWfczAdEwThaVyD0fXO50=,22.0,32.0,male,9.0,19,0,0,2831,True,...,0,0,0,0,0,0,0,0,0,False
5,++/ZHqwUNa7U21Qz+zqteiXlZapxey86l6eEorrak/g=,13.0,39.0,male,3.0,21,0,0,3129,True,...,0,0,0,0,0,0,0,0,0,False
6,++0/NopttBsaAn6qHZA2AWWrDg7Me7UOMs1vsyo4tSI=,1.0,0.0,missing,7.0,12,0,0,1788,True,...,0,0,0,0,0,0,0,0,0,False
7,++0BJXY8tpirgIhJR14LDM1pnaRosjD1mdO1mIKxlJA=,1.0,0.0,missing,9.0,16,0,0,2563,False,...,0,0,0,0,0,0,0,0,0,False
8,++0EzISdtKY48Z0GY62jer/LFQwrNIAbADdtU5xStGY=,1.0,0.0,missing,3.0,6,0,0,1073,False,...,0,0,0,0,0,0,0,0,0,False
9,++0wqjjQge1mBBe5r4ciHGKwtF/m322zkra7CK8I+Mw=,NaN,NaN,NaN,NaN,16,0,0,1584,True,...,0,0,0,0,0,0,0,0,0,False


In [3]:
data.isnull().any()

msno                   False
city                    True
bd                      True
gender                  True
registered_via          True
no_of_transactions     False
no_of_cancellations    False
cancelled              False
total_amount_paid      False
is_auto_renew          False
list_price_0           False
list_price_99          False
list_price_100         False
list_price_119         False
list_price_129         False
list_price_149         False
list_price_150         False
list_price_180         False
list_price_298         False
list_price_477         False
list_price_536         False
list_price_894         False
list_price_1788        False
list_price_1           False
list_price_10          False
list_price_15          False
list_price_35          False
list_price_50          False
list_price_70          False
list_price_105         False
list_price_120         False
list_price_126         False
list_price_131         False
list_price_134         False
list_price_143

In [4]:
cols = ['city', 'gender', 'registered_via']
for col in cols:
    data[col] = data[col].fillna('missing')

In [5]:
data['bd'] = data['bd'].fillna(0)

In [6]:
data['bd'][(data['bd'] < 0)].count()

72

In [7]:
data.loc[data['bd'] < 0, 'bd'] = 0

In [8]:
data['gender'].unique()

array(['missing', 'male', 'female'], dtype=object)

In [9]:
for col in ['city','registered_via', 'gender']:
    data[col] = data[col].astype('category')

In [10]:
data['is_auto_renew'].unique()

array([ True, False], dtype=bool)

In [17]:
data.loc[data['is_auto_renew'] == False, 'is_auto_renew'] = 0
data.loc[data['is_auto_renew'] == True, 'is_auto_renew'] = 1

# data['is_auto_renew'][data['is_auto_renew'] == False] = 0
# data['is_auto_renew'][data['is_auto_renew'] == True] = 1

In [21]:
data.loc[data['is_churn'] == False, 'is_churn'] = 0
data.loc[data['is_churn'] == True, 'is_churn'] = 1

# data['is_churn'][data['is_churn'] == False] = 0
# data['is_churn'][data['is_churn'] == True] = 1

In [11]:
data.head()

,msno,city,bd,gender,registered_via,no_of_transactions,no_of_cancellations,cancelled,total_amount_paid,is_auto_renew,...,list_price_596,list_price_699,list_price_799,list_price_930,list_price_1000,list_price_1150,list_price_1200,list_price_1599,list_price_2000,is_churn
0,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,1,0.0,missing,7,4,0,0,396,True,...,0,0,0,0,0,0,0,0,0,False
1,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,15,26.0,male,9,19,0,0,2831,True,...,0,0,0,0,0,0,0,0,0,False
2,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,1,0.0,missing,7,26,0,0,3874,True,...,0,0,0,0,0,0,0,0,0,False
3,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,18,21.0,male,7,12,0,0,1788,True,...,0,0,0,0,0,0,0,0,0,False
4,++/UDNo9DLrxT8QVGiDi1OnWfczAdEwThaVyD0fXO50=,22,32.0,male,9,19,0,0,2831,True,...,0,0,0,0,0,0,0,0,0,False


*** Now, we're loading our classifiers and further pre-processers ***

In [12]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, precision_recall_curve
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA

In [13]:
le = LabelEncoder()
cols = ['gender', 'city', 'is_auto_renew', 'is_churn', 'registered_via']
for col in cols:
    data[col] = le.fit_transform(data[col])

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 992931 entries, 0 to 992930
Data columns (total 54 columns):
msno                   992931 non-null object
city                   992931 non-null int64
bd                     992931 non-null float64
gender                 992931 non-null int64
registered_via         992931 non-null int64
no_of_transactions     992931 non-null int64
no_of_cancellations    992931 non-null int64
cancelled              992931 non-null int64
total_amount_paid      992931 non-null int64
is_auto_renew          992931 non-null int64
list_price_0           992931 non-null int64
list_price_99          992931 non-null int64
list_price_100         992931 non-null int64
list_price_119         992931 non-null int64
list_price_129         992931 non-null int64
list_price_149         992931 non-null int64
list_price_150         992931 non-null int64
list_price_180         992931 non-null int64
list_price_298         992931 non-null int64
list_price_477         992931 n

In [15]:
for col in ['city','registered_via', 'gender']:
    data[col] = data[col].astype('category')

In [16]:
X, y = data.drop(['is_churn', 'msno'], axis=1), data['is_churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

In [17]:
lgr = LogisticRegression()
lgr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [18]:
y_pred = lgr.predict(X_test)

In [20]:
print 'Logistic Regression Test error rate:', 1 - accuracy_score(y_test, y_pred)

 Logistic Regression Test error rate: 0.0683176642983


**  Our error rate on the test set is relatively low as expected. We will run our model on the hold-out set to confirm our model accuracy.**

In [24]:
y_proba = lgr.predict_proba(X_test)